In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython import display

In [ ]:
class rbc_parser:
    def __init__(self):
        self.session = requests.Session()


    def _get_url(self, param_dict: dict) -> str:
        """
        Возвращает URL для запроса json таблицы со статьями
        """
        url = (
            f"https://www.rbc.ru/search/ajax/?"
            f"project={param_dict['project']}&"
            f"category={param_dict['category']}&"
            f"dateFrom={param_dict['dateFrom']}&"
            f"dateTo={param_dict['dateTo']}&"
            f"page={param_dict['page']}&"
            f"query={param_dict['query']}&"
            f"material={param_dict['material']}"
        )
        return url


    def _get_search_table(self, param_dict: dict,
                          include_text: bool = True) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей

        include_text: bool
        ### Если True, статьи возвращаются с текстами
        """
        url = self._get_url(param_dict)
        response = self.session.get(url)

        search_table = pd.DataFrame(response.json()['items'])

        if include_text and not search_table.empty:
            get_text = lambda x: self._get_article_data(x['fronturl'])
            search_table[['overview', 'text']] = search_table.apply(get_text,
                                                                    axis=1).tolist()

        if 'publish_date_t' in search_table.columns:
            search_table.sort_values('publish_date_t', ignore_index=True)

        return search_table


    def _iterable_load_by_page(self, param_dict):
        param_copy = param_dict.copy()
        results = []

        result = self._get_search_table(param_copy)
        results.append(result)

        while not result.empty:
            param_copy['page'] = str(int(param_copy['page']) + 1)
            result = self._get_search_table(param_copy)
            results.append(result)

        results = pd.concat(results, axis=0, ignore_index=True)

        return results


    def _get_article_data(self, url: str):
        """
        Возвращает описание и текст статьи по ссылке
        """
        response = self.session.get(url)

        soup = bs(response.text, features="lxml")

        div_overview = soup.find('div', {'class': 'article__text__overview'})

        if div_overview:
            overview = div_overview.text.replace('<br />','\n').strip()
        else:
            overview = None

        all_paragraphs = soup.find_all('p')

        filtered_paragraphs = [
            p for p in all_paragraphs
            if not p.find_parent('div', class_='article__special_container')
            and not p.find_parent('div', class_='showcase-collection__subtitle')
            and not p.find_parent('div', class_='showcase-collection-card__text')
            and not p.find_parent('div', class_='showcase-collection__footer')
        ]

        if filtered_paragraphs:
            text = ' '.join(
                p.text.replace('<br />', '\n').strip()
                for p in filtered_paragraphs
                )
        else:
            text = None

        return overview, text


    def get_articles(self,
                     param_dict,
                     time_step = 1,
                     save_every = 5,
                     save_excel = True) -> pd.DataFrame:
        """
        Функция для скачивания статей интервалами через каждые time_step дней
        Делает сохранение таблицы через каждые save_every * time_step дней

        param_dict: dict
        ### Параметры запроса
        ###### project - раздел поиска, например, rbcnews
        ###### category - категория поиска, например, TopRbcRu_economics
        ###### dateFrom - с даты
        ###### dateTo - по дату
        ###### query - поисковой запрос (ключевое слово), например, РБК
        ###### page - смещение поисковой выдачи (с шагом 20)

        ###### Deprecated:
        ###### offset - смещение поисковой выдачи
        ###### limit - лимит статей, максимум 100
        """
        param_copy = param_dict.copy()
        time_step = timedelta(days=time_step)

        dateFrom = datetime.strptime(param_copy['dateFrom'], '%d.%m.%Y')
        dateTo = datetime.strptime(param_copy['dateTo'], '%d.%m.%Y')

        if dateFrom > dateTo:
            raise ValueError('dateFrom should be less than dateTo')

        out = pd.DataFrame()
        save_counter = 0

        while dateFrom <= dateTo:
            param_copy['dateTo'] = (dateFrom + time_step).strftime("%d.%m.%Y")
            if dateFrom + time_step > dateTo:
                param_copy['dateTo'] = dateTo.strftime("%d.%m.%Y")

            print('Parsing articles from ' + param_copy['dateFrom'] +  ' to ' + param_copy['dateTo'])

            out = pd.concat([out, self._iterable_load_by_page(param_copy)], axis=0, ignore_index=True)

            dateFrom += time_step + timedelta(days=1)

            param_copy['dateFrom'] = dateFrom.strftime("%d.%m.%Y")

            save_counter += 1
            if save_counter == save_every:
                display.clear_output(wait=True)
                out.to_excel(f"/tmp/rbc_{param_dict['dateFrom']}_{param_dict['dateTo']}_checkpoint_table.xlsx")
                print('Checkpoint saved!')
                save_counter = 0

        if save_excel:
            out.to_excel(f"rbc_{param_dict['dateFrom']}_{param_dict['dateTo']}.xlsx")
        print("Finish")

        return out

### РБК

* __project__ - проекты РБК. Возможные значения: ["rbcnews", "rbctv", "rbcstyle", "sport", "realty", "crypto", "autonews", "quote", "bc3", "trends"]

* __category__ - рубрики: ["TopRbcRu_economics", "TopRbcRu_auto", "TopRbcRu_business", "TopRbcRu_money", "TopRbcRu_realty", "TopRbcRu_society", "TopRbcRu_politics", "TopRbcRu_own_business", "TopRbcRu_specials", "TopRbcRu_technology_and_media", "TopRbcRu_finances"]

* __material__ - материалы: ["video", "quiz", "interview", "research", "card", "opinion", "multimedia", "short_news", "olympics_online", "online", "investigation", "rating", "article_specproject", "article", "story"]

* __dateFrom__ - с даты

* __dateTo__ - по дату

* __page__ - смещение запроса (с шагом 20)

* __Deprecated__:

    * __offset__ - смещение поисковой выдачи

    * __limit__ - лимит запроса, максимум 20

_Чтобы не специфировать параметр, оставляем поле пустым_

In [ ]:
# Задаем параметры запроса и складываем в param_dict

use_parser = "РБК"

query = 'РБК'
project = "rbcnews"
category = "TopRbcRu_economics"
material = ""
dateFrom = '2025-01-01'
dateTo = "2025-01-07"
page = 0

if use_parser == "РБК":
    param_dict = {
        'query'   : query,
        'project' : project,
        'category': category,
        'dateFrom': datetime.
        strptime(dateFrom, '%Y-%m-%d').
        strftime('%d.%m.%Y'),
        'dateTo'  : datetime.
        strptime(dateTo, '%Y-%m-%d').
        strftime('%d.%m.%Y'),
        'page'   : str(page),
        'material': material
    }

print(use_parser, "- param_dict:", param_dict)

РБК - param_dict: {'query': 'РБК', 'project': 'rbcnews', 'category': 'TopRbcRu_economics', 'dateFrom': '01.01.2025', 'dateTo': '07.01.2025', 'page': '0', 'material': ''}


In [ ]:
# Пример того, как выглядит json таблица запроса по параметрам.
# Действует ограничение в 20 статей на 1 запрос (параметром limit)

assert use_parser == "РБК"
parser = rbc_parser()
tbl = parser._get_search_table(
    param_dict,
    include_text = True # Парсить текст статей
)
print(len(tbl))
tbl.head()

20


,id,project,project_nick,type,category,title,body,publish_date,publish_date_t,fronturl,picture,badge,pay_option,data,_score,overview,text
0,677d8f6e9a7947fde05968ac,РБК,rbcnews,short_news,Политика,Умеров назначил своим советником уволенного эк...,,2025-01-07T23:58:35+03:00,1736283515,https://www.rbc.ru/rbcfreenews/677d8f6e9a7947f...,None,None,free,None,1,None,Министр обороны Украины Рустем Умеров назначил...
1,677d892c9a7947bb5f46e9dd,РБК,rbcnews,short_news,Общество,Цены на подержанные часы Rolex побили минимум ...,,2025-01-07T23:39:12+03:00,1736282352,https://www.rbc.ru/rbcfreenews/677d892c9a7947b...,None,None,free,None,1,None,В 2024 году цены на самые популярные подержанн...
2,677d8dda9a79474ed21b3e4d,РБК,rbcnews,article,Политика,Трамп заявил о планах переименовать Мексиканск...,Избранный президент США также вновь раскритико...,2025-01-07T23:29:25+03:00,1736281765,https://www.rbc.ru/politics/07/01/2025/677d8dd...,https://s0.rbk.ru/v6_top_pics/media/img/9/45/3...,None,free,None,1,Избранный президент США также вновь раскритико...,"Избранный президент США Дональд Трамп заявил, ..."
3,677d83189a794754cc2f7ccc,РБК,rbcnews,short_news,Политика,В Днепре мужчина ударил ножом сотрудника ТЦК,,2025-01-07T23:04:09+03:00,1736280249,https://www.rbc.ru/rbcfreenews/677d83189a79475...,None,None,free,None,1,None,Украинец нанес ножевое ранение сотруднику терр...
4,677d80a39a794750e4572bf5,РБК,rbcnews,article,Спорт,Российский лыжник сменил спортивное гражданств...,Двукратный чемпион Универсиады 2017 года Валер...,2025-01-07T22:54:15+03:00,1736279655,https://www.rbc.ru/sport/07/01/2025/677d80a39a...,https://s0.rbk.ru/v6_top_pics/media/img/7/53/3...,None,free,None,1,Двукратный чемпион Универсиады 2017 года Валер...,Российский лыжник Валерий Гонтарь сменил спорт...


In [ ]:
tbl['text'][3]

'Президент США Дональд Трамп считает, что Россия затягивает переговоры о всеобъемлющем прекращении огня, в его администрации рассматривают возможность «агрессивных санкций», в том числе против российского «теневого флота», сообщила журналистка Fox News Джеки Хайнрих в соцсети X со ссылкой на источник. По словам собеседника, Трамп разочарован действиями российского коллеги Владимира Путина. «Мне сказали, что соблюдение санкций сейчас находится на низком уровне\xa0— по неофициальным оценкам, примерно 3 из 10. И российский «теневой флот», перевозящий нелегальную нефть через Балтийское море, будет легкой мишенью для усиления давления»,\xa0— написала журналистка в соцсети. Сам американский лидер на днях пригрозил, что введет вторичные пошлины на всю нефть из России, если не удастся заключить перемирие на Украине по вине Москвы. Он пообещал, что «25-процентный тариф на нефть и другие продукты, продаваемые в Соединенных Штатах, вторичные тарифы» будут введены в течение месяца, если не удастся

In [ ]:
%%time

# Пример работы программы итеративного сбора большого количества текстов статей
table = parser.get_articles(
    param_dict=param_dict,
    time_step = 5, # Шаг - 7 дней, можно больше, но есть риск отсечения статей в неделях, где статей больше 100
    save_every = 1, # Сохранять чекпойнт каждые x шагов
    save_excel = True # Сохранить итоговый файл
)
print(len(table))
table.head()

Checkpoint saved!
Finish
713
CPU times: user 1min 10s, sys: 394 ms, total: 1min 10s
Wall time: 6min 12s


,id,project,project_nick,type,category,title,body,publish_date,publish_date_t,fronturl,picture,badge,pay_option,data,_score,overview,text
0,677c3c699a79471ec81f21c2,РБК,rbcnews,article,Общество,Взрыв газа произошел в жилом доме в Петрозаводске,На данный момент известно о трех пострадавших....,2025-01-06T23:27:29+03:00,1736195249,https://www.rbc.ru/society/06/01/2025/677c3c69...,None,None,free,None,1,На данный момент известно о трех пострадавших....,Взрыв газа произошел в одной из квартир пятиэт...
1,677c3b7d9a79471ec81f21c0,РБК,rbcnews,short_news,Общество,Шесть снегоходов с людьми провалились под лед ...,,2025-01-06T23:25:20+03:00,1736195120,https://www.rbc.ru/rbcfreenews/677c3b7d9a79471...,None,None,free,None,1,None,В Прионежском районе Карелии шесть снегоходов ...
2,677c37e09a7947030de47f4b,РБК,rbcnews,article,Политика,Во Франции сообщили о «десятках» дезертиров ср...,О массовом дезертирстве из бригады ВСУ «Анна К...,2025-01-06T23:16:17+03:00,1736194577,https://www.rbc.ru/politics/06/01/2025/677c37e...,https://s0.rbk.ru/v6_top_pics/media/img/0/78/3...,None,free,None,1,О массовом дезертирстве из бригады ВСУ «Анна К...,"Министерство обороны Франции подтвердило, что ..."
3,677c2f609a7947475c99fcce,РБК,rbcnews,article,Политика,США сняли санкции с операций по энергоресурсам...,"О том, что США смягчат санкции против Сирии, н...",2025-01-06T22:42:45+03:00,1736192565,https://www.rbc.ru/politics/06/01/2025/677c2f6...,https://s0.rbk.ru/v6_top_pics/media/img/5/68/3...,None,free,None,1,"О том, что США смягчат санкции против Сирии, н...",Соединенные Штаты вывели из-под санкций ряд фи...
4,677c2cc49a7947b695efd435,РБК,rbcnews,article,Политика,"Bild узнал, что глава сирийского Минюста приго...","В Сети появились видео 10-летней давности, на ...",2025-01-06T22:28:57+03:00,1736191737,https://www.rbc.ru/politics/06/01/2025/677c2cc...,None,None,free,None,1,"В Сети появились видео 10-летней давности, на ...","Министр юстиции Сирии Шади Аль-Вайси, назначен..."
